In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import h5py 

import torch
import mlflow

import thisquakedoesnotexist
from thisquakedoesnotexist.models import gan
from thisquakedoesnotexist.utils.data_utils import SeisData

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
sns.set()

color_palette = sns.color_palette('dark')
colors = [color_palette[3], color_palette[7], color_palette[0], color_palette[1], color_palette[2], color_palette[4], color_palette[5], color_palette[6], color_palette[8], color_palette[9]]
sns.set_palette(sns.color_palette(colors))
# mpl.use('Agg')

In [ ]:
logged_model = '../../mlruns/893763428974995232/8a26d9ee666e4739bae9cd68dfa2e466/artifacts/thisquakedoesnotexist/data/output_8a26d9ee/model_epoch_00030'


loaded_model = mlflow.pytorch.load_model(logged_model)

In [ ]:
data = np.random.random(50*120)

In [ ]:
filled = np.nan_to_num(data)

In [ ]:
data

In [ ]:
file = h5py.File('../data/japan/downsampled.h5')
file['waveforms'][0].shape

In [ ]:
waveforms = np.load('../data/japan/waveforms.npy')
waveforms.shape


In [ ]:
waveforms.shape

for i in range(10):
    print(waveforms[i].shape)
    print(waveforms[i, 1:].max(), waveforms[i, 0])

In [ ]:
file2 = h5py.File('../data/japan/wforms_GAN_input_v20220805.h5')
file2['waveforms'][1, :, 1]

for i in range(10):
    print(np.nan_to_num(file2['waveforms'][1, :, i], 0).max())

In [ ]:
data_file = '../data/japan/waveforms.npy'
attr_file = '../data/japan/attributes.csv'
batch_size = 10
sample_rate = 20

condv_names = ['dist', 'mag'] # , 'vs30']
nbins_dict = {
    'dist': 30,
    'mag': 30,
    # 'vs30': 20,
}

f = np.load(data_file)
num_samples = len(f)
del f

# get all indexes
ix_all = np.arange(num_samples)

sdat_train = SeisData(
        data_file=data_file,
        attr_file=attr_file,
        batch_size=batch_size,
        sample_rate=sample_rate,
        v_names=condv_names,
        #nbins_d=nbins_dict,
        isel=ix_all,
    )



In [ ]:
def plot_waves_1C(dataset, ws, i_vg, args, t_max=50.0, ylim=None, fig_file=None, Np=12, fig_size = (10,15), stitle=None, show_fig=False):
    """
        Make plot of waves
        shape of the data has the form -> (Nobs, Nt)
        :param ws_p: numpy.ndarray of shape (Nobs, Nt)
        :param Np: number of 3C waves to show
        :param color: Color of the plot
        :param t_max: max time for the plot in sec
        :return: show a panel with the waves

        """
    nt = int(t_max / args.time_delta)
    if ws.shape[0] < Np:
        # in case there are less observations than points to plot
        Np = ws.shape[0]

    ws_p = ws[:Np, :]

    # select waves to plot
    tt = args.time_delta * np.arange(nt)
    plt.figure()
    fig, ax = plt.subplots(Np, 1, sharex='col', 
                            gridspec_kw={'hspace': 0.4, 'wspace': 0.05},
                            figsize=fig_size)

    fig.add_subplot(111, frameon=False, ylabel='Log Amplitude')
    plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    plt.grid(False)
    
    plt.xlabel("Time [s]")
   #plt.ylabel("Log-Amplitude")

    for ik in range(Np):
        wf = ws_p[ik,:]

        dist = dataset.to_real(i_vg[0][ik], 'dist').cpu().numpy()[0]
        mag = dataset.to_real(i_vg[1][ik], 'mag').cpu().numpy()[0]
        # vs30 = dataset.to_real(i_vg[2][ik], 'vs30').cpu().numpy()[0]

        ax[ik].plot(tt, wf, lw=0.5)

        low, high = ax[ik].get_ylim()
        bound = max(abs(low), abs(high))
        ax[ik].set_ylim(-bound, bound)
        ax[ik].set_title(f'Dist: {dist:.1f}, Mag: {mag:.1f}') #, Vs30: {vs30:.1f}')

        if ylim is not None:
            ax[ik].set_ylim(ylim)

    for ax in ax.flat:
        ax.label_outer()

    if stitle is not None:
        fig.suptitle(stitle, fontsize=12)

    if show_fig:
        fig.show()

    if fig_file is not None:
        fformat = fig_file.split('.')[-1]
        print('saving:', fformat)
        fig.savefig(fig_file, format=fformat)
        if not show_fig:
            plt.clf()
    else:
        fig.show()
    plt.clf()
    plt.cla()
    plt.close()


In [ ]:
noise_dim = 100
num_plots = 20

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f"Running on device: {device}")

In [ ]:
class rand_noise(object):
    def __init__(self, nchan, size, device=None):
        self.nchan = nchan
        self.size = size
        self.device = device
    def sample(self, Nb, ):
        ur = torch.randn(Nb, self.nchan, self.size, device=self.device)
        return ur

grf = rand_noise(1, noise_dim, device=device)

In [ ]:
output_dir = '../data/output/tmp'

In [ ]:
fig_dir = os.path.join(output_dir, 'figs')
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

In [ ]:
G = loaded_model

G.eval();

In [ ]:
def plot_wave(waveform):
    t_max = 50
    dt = 0.05
    Nt = int(t_max / dt)

    ws_p = waveform
    # select waves to plot
    tt = dt * np.arange(Nt)
    plt.figure(figsize=(16, 8))
    
    plt.plot(tt, ws_p, lw=0.5)
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.show()

In [ ]:
i_vg = sdat_train.get_rand_cond_v()
i_vg = [torch.from_numpy(i_v).float().cuda() for i_v in i_vg]

i_vg

In [ ]:
print(sdat_train.cnorms.shape)
print(sdat_train.df_meta)

sdat_train.df_meta.loc[0, 'dist']

In [ ]:

plt.figure(figsize=(15, 10))
plt.yscale('log')
plt.scatter(np.arange(len(sdat_train.cnorms)), sdat_train.cnorms, lw=0.5)
plt.title('Max Amplitude Distribution of Dataset (Magnitude Increases From Left to Right)')
plt.show()


In [ ]:
# pd.DataFrame(sdat_train.wfs).describe()

num_plots = 6

plt.figure()
fig, ax = plt.subplots(num_plots, 1, figsize=(10, 15))

tt = 0.05 * np.arange(1000)

for i in range(num_plots):
    waveform_i = 10000 * i
    ax[i].plot(tt, sdat_train.wfs[waveform_i] * sdat_train.cnorms[waveform_i], lw=0.5, label=f'Signal Number {waveform_i}')
    ax[i].set_title(f"Dist: {sdat_train.df_meta.loc[waveform_i, 'dist']:.1f}, Mag: {sdat_train.df_meta.loc[waveform_i, 'mag']:.1f}") #, vs30: {sdat_train.df_meta.loc[waveform_i, 'vs30']:.1f}")
    # ax[0].set_ylim([minsignal, maxsignal])
    ax[i].set_xlabel('Time [s]')
    ax[i].set_ylabel('Amplitude')
    ax[i].legend()

    plt.subplots_adjust(hspace=1.0)
plt.savefig('real_data_samples.pdf')

In [ ]:
dist_max = sdat_train.vc_max['dist']
mag_max = sdat_train.vc_max['mag']
# vs30_max = sdat_train.vc_max['vs30']

print(f'dist max: {dist_max}\nmag mag: {mag_max}') #\nvs30 max: {vs30_max}')

In [ ]:
samples = 10

dist = 70
mag = 4.5
# vs30 = sdat_train.df_meta['vs30'].mean() #vs30_max

vc_list = [
    dist / dist_max * torch.ones(samples, 1).cuda(),
    mag / mag_max * torch.ones(samples, 1).cuda(),
    # vs30 /vs30_max * torch.ones(samples, 1).cuda(),
]

grf = rand_noise(1, noise_dim, device=device)
z = grf.sample(samples)

G.eval()
x_g, x_scaler = G(z, *vc_list)

# x_g = G(z, distv)
x_g = x_g.squeeze().detach().cpu()
x_scaler = x_scaler.squeeze().detach().cpu()
# x_g = x_g.detach().cpu()


good_samples = []
for wf, scaler in zip(x_g, x_scaler):
    tv = np.sum(np.abs(np.diff(wf)))
    # If generator sample fails to generate a seismic signal, skip it
    # threshold value is emperically chosen
    if tv < 40:
        continue
    
    wf = wf * scaler
    
    good_samples.append(wf)

In [ ]:
for i in range(5):
    x = good_samples[i].detach().cpu().numpy()
    plot_wave(x)


In [ ]:
samples = 1000

dist = 60
mag = 6
# vs30 = sdat_train.df_meta['vs30'].mean()

vc_list = [
    dist / dist_max * torch.ones(samples, 1).cuda(),
    mag / mag_max * torch.ones(samples, 1).cuda(),
    # vs30 /vs30_max * torch.ones(samples, 1).cuda(),
]            

grf = rand_noise(1, noise_dim, device=device)
z = grf.sample(samples)

G.eval()
x_g, x_scaler = G(z, *vc_list)

x_g = x_g.squeeze().detach().cpu()
x_scaler = x_scaler.squeeze().detach().cpu()

x_g = x_g * x_scaler

In [ ]:
plot_wave(x_g[674])

In [ ]:
random_data = grf.sample(samples)
syn_data, syn_scaler = G(random_data, *vc_list)
syn_data = syn_data.squeeze()
syn_data = np.array(syn_data.detach().cpu())

scaler = syn_data[:, 0]
scaler = scaler[:, np.newaxis]

syn_data = syn_data * scaler
print(syn_data)

In [ ]:
dist = 70
mag = 5.8
# vs30 = sdat_train.df_meta['vs30'].mean()

vc_list = [
    dist / dist_max * torch.ones(samples, 1).cuda(),
    mag / mag_max * torch.ones(samples, 1).cuda(),
    # vs30 /vs30_max * torch.ones(samples, 1).cuda(),
]    
random_data = grf.sample(samples)
syn_data, syn_scaler = G(random_data, *vc_list)
syn_data = syn_data.squeeze()
syn_data = np.array(syn_data.detach().cpu())

scaler = syn_data[0]
scaler = scaler[:, np.newaxis]
syn_data = syn_data * syn_data[0]


In [ ]:
fig = plt.figure()#figsize=figsize,)

dt = 0.05
samples = 300

for dist in [40, 60, 80, 100, 120, 140, 160]:
    mag = 6
    # vs30 = sdat_train.df_meta['vs30'].mean()

    vc_list = [
        dist / dist_max * torch.ones(samples, 1).cuda(),
        mag / mag_max * torch.ones(samples, 1).cuda(),
        # vs30 /vs30_max * torch.ones(samples, 1).cuda(),
    ]    
    random_data = grf.sample(samples)
    syn_data, syn_scaler = G(random_data, *vc_list)
    syn_data = syn_data.squeeze()
    syn_data = np.array(syn_data.detach().cpu())
    syn_scaler = syn_scaler.detach().cpu().numpy()
    
    syn_data = syn_data * syn_scaler

    synthetic_data_log = np.log(np.abs(np.array(syn_data + 1e-10)))
    sd_mean = np.mean(synthetic_data_log, axis=0)

    y = np.exp(sd_mean)

    Nt = synthetic_data_log.shape[1]
    tt = dt * np.arange(0, Nt)
    plt.semilogy(tt, y, '-' , label=f'Dist: {dist}km, Mag: {mag}', alpha=0.8, lw=0.5)

plt.legend()
plt.show()
# [40.00, 54.01], [54.01, 68.03], [68.03, 82.04], [82.04, 96.05], [96.05, 110.06], [110.06, 124.07],


In [ ]:
fig = plt.figure()#figsize=figsize,)

dt = 0.05
samples = 300

for mag in [4.5, 5.0, 5.5, 6.0, 6.5, 7.0]:
    dist = 70
    # mag = 5.8
    

    vc_list = [
        dist / dist_max * torch.ones(samples, 1).cuda(),
        mag / mag_max * torch.ones(samples, 1).cuda(),
        # vs30 / vs30_max * torch.ones(samples, 1).cuda(),
    ]    
    random_data = grf.sample(samples)
    syn_data, syn_scaler = G(random_data, *vc_list)
    syn_data = syn_data.squeeze().detach().cpu().numpy()

    syn_data = syn_data * syn_scaler.detach().cpu().numpy()
    
    synthetic_data_log = np.log(np.abs(np.array(syn_data + 1e-10)))
    sd_mean = np.mean(synthetic_data_log, axis=0)

    y = np.exp(sd_mean)

    Nt = synthetic_data_log.shape[1]
    tt = dt * np.arange(0, Nt)
    plt.semilogy(tt, y, '-' , label=f'Dist: {dist}km, Mag: {mag}', alpha=0.8, lw=0.5)

plt.legend()
plt.show()
# [40.00, 54.01], [54.01, 68.03], [68.03, 82.04], [82.04, 96.05], [96.05, 110.06], [110.06, 124.07],


In [ ]:
def get_waves_real_bin(s_dat, distbs, mbs, verbose=0):
    # get dataframe with attributes
    df = s_dat.df_meta
    # get waves
    wfs = s_dat.wfs
    cnorms = s_dat.cnorms
    # print(df.shape)
    # select bin of interest
    ix = ((distbs[0] <= df['dist']) & (df['dist'] < distbs[1]) &
          (mbs[0] <= df['mag']) & (df['mag'] <= mbs[1]) #&
          #(vsb[0] <= df['vs30']) & (df['vs30'] < vsb[1]))
    )

    # get normalization coefficients
    df_s = df[ix]
    # get waveforms
    ws_r = wfs[ix, :]
    c_r = cnorms[ix]
    n_obs = ix.sum()

    means = {'dist': df_s['dist'].mean(),
             'mag': df_s['mag'].mean(),
             #'vs30': df_s['vs30'].mean(),
             }
    if verbose:
        print('# observations', n_obs)
        print(f"Mag range: [{df_s['mag'].min():.2f}, {df_s['mag'].max():.2f})")
        print(f"Mag mean: {df_s['mag'].mean():.2f}")

        print(f"Dist range: [{df_s['dist'].min():.2f}, {df_s['dist'].max():.2f})")
        print(f"Mag mean: {df_s['dist'].mean():.2f}")

        # print(f"Vs30 range: [{df_s['vs30'].min():.2f}, {df_s['vs30'].max():.2f})")
        # print(f"Vs30 mean: {df_s['vs30'].mean():.2f}")
      
    return (ws_r, c_r, means, n_obs)

In [ ]:
dist_min = sdat_train.vc_min['dist'] - 1e-5
dist_max = sdat_train.vc_max['dist'] + 1e-5
dist_step_size = (dist_max - dist_min) / 10.0
dist_bins = np.arange(dist_min, dist_max + dist_step_size/2.0, step=dist_step_size)

mag_min = sdat_train.vc_min['mag'] - 1e-5
mag_max = sdat_train.vc_max['mag'] + 1e-5
mag_step_size = (mag_max - mag_min) / 10.0
mag_bins = np.arange(mag_min, mag_max + mag_step_size/2.0, step=mag_step_size)

# vs30_min = sdat_train.vc_min['vs30'] - 1e-5
# vs30_max = sdat_train.vc_max['vs30'] + 1e-5
# vs30_step_size = (vs30_max - vs30_min)
# vs30_bins = np.arange(vs30_min, vs30_max + vs30_step_size/2.0, step=vs30_step_size)

In [ ]:
def plot_real_syn_bucket(wfs, c_norms, means, n_obs, dist_border, mag_border):
    # wfs, c_norms, means, n_obs = get_waves_real_bin(sdat_train, dist_border, mag_border, vs30_bins, verbose=3)

    c_norms = c_norms.reshape(-1, 1)
    real_data = np.log(np.abs(wfs * c_norms) + 1e-10)

    rd_25 = np.exp(np.percentile(real_data, 25, axis=0))
    rd_75 = np.exp(np.percentile(real_data, 75, axis=0))

    real_data_mean = np.exp(real_data.mean(axis=0))
    # real_data_median = np.exp(np.median(real_data, axis=0))
    
    real_data = np.exp(real_data)

    samples = n_obs
    dt = 0.05

    dist = means['dist']
    mag = means['mag']
    # vs30 = means['vs30']

    vc_list = [
        dist / dist_max * torch.ones(samples, 1).cuda(),
        mag / mag_max * torch.ones(samples, 1).cuda(),
        # vs30 / vs30_max * torch.ones(samples, 1).cuda(),
    ]

    random_data = grf.sample(samples)
    syn_data, syn_scaler = G(random_data, *vc_list)
    syn_data = syn_data.squeeze().detach().cpu().numpy()

    syn_data = syn_data * syn_scaler.detach().cpu().numpy()

    synthetic_data_log = np.log(np.abs(syn_data + 1e-10))
    sd_mean = np.mean(synthetic_data_log, axis=0)
    sd_mean = np.exp(sd_mean)

    sd_25 = np.exp(np.percentile(synthetic_data_log, 25, axis=0))
    sd_75 = np.exp(np.percentile(synthetic_data_log, 75, axis=0))

    Nt = synthetic_data_log.shape[1]
    tt = dt * np.arange(0, Nt)

    fig = plt.figure(figsize=(16, 8))
    plt.semilogy(tt, sd_mean, '-' , label=f'Synthetic Data', alpha=0.8, lw=0.5)
    plt.fill_between(tt, sd_75, sd_25, alpha=0.1)
    plt.semilogy(tt, real_data_mean, '-' , label=f'Real Data', alpha=0.8, lw=0.5)
    # plt.semilogy(tt, real_data_median, '-' , label=f'Real Data Median', alpha=0.8, lw=0.5)
    plt.fill_between(tt, rd_75, rd_25, alpha=0.1)

    plt.ylabel('Log-Amplitude')
    plt.xlabel('Time [s]')
    plt.legend()
    plt.title(f"Obs: {n_obs}, Dist: [{dist_border[0]:.1f},{dist_border[1]:.1f}] km, Mag: [{mag_border[0]:.1f},{mag_border[1]:.1f}]")
    plt.savefig(f'tmp/dist_[{dist_border[0]:.1f},{dist_border[1]:.1f}]_mag_[{mag_border[0]:.1f},{mag_border[1]:.1f}].pdf')


In [ ]:
def calc_mean_distances(wfs, c_norms, means, n_obs, dist_border, mag_border):
    c_norms = c_norms.reshape(-1, 1)
    real_data = wfs * c_norms
    real_data_mean = np.mean(real_data, axis=0)
    
    dist = means['dist']
    mag = means['mag']
    # vs30 = means['vs30']

    vc_list = [
        dist / dist_max * torch.ones(samples, 1).cuda(),
        mag / mag_max * torch.ones(samples, 1).cuda(),
        # vs30 / vs30_max * torch.ones(samples, 1).cuda(),
    ]

    random_data = grf.sample(samples)
    syn_data, syn_scaler = G(random_data, *vc_list)
    syn_data = syn_data.squeeze().detach().cpu().numpy()

    syn_data = syn_data * syn_scaler.detach().cpu().numpy()

    sd_mean = np.mean(syn_data, axis=0)

    l2_dist = np.sum(np.abs(real_data_mean - sd_mean)) / len(sd_mean)
    mse = np.sum((real_data_mean - sd_mean)**2) / len(sd_mean)
    # print("L2: ", l2_dist)
    # print("MSE: ", mse)
    return (l2_dist, mse)

In [ ]:
dist_bin_centers = []
mag_bin_centers = []

for i in range(10):
    dist_border = [dist_bins[i], dist_bins[i+1]]
    mag_border = [mag_bins[i], mag_bins[i+1]]
    wfs, c_norms, means, n_obs = get_waves_real_bin(sdat_train, dist_border, mag_border) #, vs30_bins)
    dist_bin_centers.append(means['dist'].round(1))
    mag_bin_centers.append(means['mag'].round(1))

In [ ]:
l2_mat = np.full((10, 10), np.nan)
mse_mat = np.full((10, 10), np.nan)

for i in range(10):
    for j in range(10):
        dist_border = [dist_bins[i], dist_bins[i+1]]
        mag_border = [mag_bins[j], mag_bins[j+1]]
        wfs, c_norms, means, n_obs = get_waves_real_bin(sdat_train, dist_border, mag_border) #, vs30_bins)
        
        if n_obs < 25:
            print(f"Bucket [{i}, {j}] only contains {n_obs} waveforms. Skipping ...")
            print(dist_border, mag_border)
            continue
        
        plot_real_syn_bucket(wfs, c_norms, means, n_obs, dist_border, mag_border)
        l2, mse = calc_mean_distances(wfs, c_norms, means, n_obs, dist_border, mag_border)
        l2_mat[i, j] = l2
        mse_mat[i, j] = mse
    

In [ ]:
plt.title('Log-L2 Distance Real-Synthetic Data')
ax = sns.heatmap(np.log(l2_mat), xticklabels=mag_bin_centers, yticklabels=dist_bin_centers)
ax.invert_yaxis()
plt.xlabel('Bin Mean Magnitude')
plt.ylabel('Bin Mean Distance [km]')
plt.show()

plt.title('Log-MSE Distance Real-Synthetic Data')
ax = sns.heatmap(np.log(mse_mat), xticklabels=mag_bin_centers, yticklabels=dist_bin_centers)
ax.invert_yaxis()
plt.xlabel('Bin Mean Magnitude')
plt.ylabel('Bin Mean Distance [km]')
plt.show()

In [ ]:
w = 50
conv_data = np.convolve(synthetic_data_log[0], np.ones(w), 'valid') / w

In [ ]:
y = np.exp(conv_data)

fig = plt.figure(figsize=(9, 6))

Nt = y.shape[0]
tt = dt * np.arange(0, Nt)
plt.semilogy(tt, y, '-')
# plt.fill_between(tt, eglup, egldn, alpha=0.2)
 

plt.show()

In [ ]:
dist_min = sdat_train.df_meta['dist'].min()
dist_max = sdat_train.df_meta['dist'].max()
dist_mean = sdat_train.df_meta['dist'].mean()

mag_min = sdat_train.df_meta['mag'].min()
mag_max = sdat_train.df_meta['mag'].max()
mag_mean = sdat_train.df_meta['mag'].mean()

# vs30_mean = sdat_train.df_meta['vs30'].mean()
# vs30_max = sdat_train.df_meta['vs30'].max()

In [ ]:
import itertools

In [ ]:
n_rows = 8
n_cols = 4
n_tot = n_rows * n_cols 

# vs30 = vs30_mean

dists = [dist_min, dist_mean, dist_max]
mags = [mag_min, mag_mean, mag_max]

for dist, mag in itertools.product(dists, mags):
    vc_list = [
            dist / dist_max * torch.ones(n_tot, 1).cuda(),
            mag / mag_max * torch.ones(n_tot, 1).cuda(),
            # vs30 / vs30_max * torch.ones(n_tot, 1).cuda(),
    ]    

    random_data = grf.sample(n_tot)
    syn_data, syn_scaler = G(random_data, *vc_list)
    syn_data = syn_data.squeeze()
    syn_data = np.array(syn_data.detach().cpu())
    syn_scaler = syn_scaler.detach().cpu().numpy()

    syn_data = syn_data * syn_scaler

    n_t = x_g.shape[1]
    tt = dt * np.arange(0, n_t)

    plt.figure()
    fig, axs = plt.subplots(n_rows, n_cols, sharex='col',
                            gridspec_kw={'hspace': 0.2, 'wspace': 0.15},
                            figsize=(36,12),
                            )

    fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    plt.grid(False)
    plt.xlabel("Time [s]")
    plt.ylabel("Amplitude", labelpad=10.0)

    for i, ax in enumerate(axs.flat):
        # ax.set_ylim((-1, 1))
        # ax.set_aspect('equal', 'box')
        ax.plot(tt, syn_data[i, :], linewidth=0.5)
        low, high = ax.get_ylim()
        bound = max(abs(low), abs(high))
        ax.set_ylim(-bound, bound)
        

    # plt.tight_layout(pad=0.2)
    fig.suptitle(f'Randomly drawn samples from Generator. Dist: {dist:.1f} km, Mag: {mag:.1f}')
    plt.show()

In [ ]:
def rolling_window(x, window_len, step_length):
    pos = 0
    while pos <= len(x) - window_len:
        yield x[pos : pos+window_len]
        pos += step_length


window_size = 20
half_wl = int(window_size / 2)

# signal = np.array(x_g[1])
signal = np.log(np.abs(np.array(x_g[0, :])))
rw = rolling_window(signal, window_size, step_length=window_size)

center = int(0.5 * window_size)
rolling_mean = []
rolling_median = []
rolling_max = []

for i, window in enumerate(rw):
    rolling_mean.append(window.mean())
    rolling_median.append(np.median(window))
    rolling_max.append(window.max())
    

locs = dt * np.arange(half_wl, len(rolling_mean) * window_size, window_size)

plt.plot(tt, signal, lw=0.5)
plt.plot(locs, rolling_mean)
plt.show()

plt.plot(tt, signal, lw=0.5)
plt.plot(locs, rolling_median)
plt.show()

plt.plot(tt, signal, lw=0.5)
plt.plot(locs, rolling_max)
plt.show()

    

In [ ]:
window_size = 20
    
# signal = np.array(x_g[1])
signal = np.log(np.abs(np.array(x_g[0, :])))
rw = rolling_window(signal, window_size, window_size)

center = int(0.5 * window_size)
rolling_max = []

for i, window in enumerate(rw):
    rolling_max.append(window.max())
    
tt = np.arange(0, 50, 0.05)
t_rm = np.linspace(0, 50, 50)
plt.plot(tt, signal, label='Signal', lw=0.5)
plt.plot(t_rm, rolling_max, label='Max Win 20')

plt.legend()
plt.show()

In [ ]:
print("Synthetic data: ")

for i in range(10):
    a = x_g[i, :]
    tv = np.sum(np.abs(np.diff(a)))
    print(tv)


plt.plot(x_g[9, :], alpha=0.5, label='Synthetic data')

dist_bin = [50, 70]
mag_bin = [5.9, 6.1]
# vs30_bin = [vs30_min, vs30_max]

(ws_r, c_r, means, n_obs) = get_waves_real_bin(sdat_train, dist_bin, mag_bin, verbose=0)
real_data = ws_r * c_r.reshape(-1, 1)

print("Real data: ")
for i in range(10):
    a = real_data[i, :]
    tv = np.sum(np.abs(np.diff(a)))
    print(tv)


plt.plot(real_data[0, :], alpha=0.5, label='Real data')
plt.legend()
plt.show()

In [ ]:
signal = np.log(np.abs(np.array(x_g[0, :])))
signal = np.array(x_g[0, :])

s_fft = np.fft.fft(signal)

s_ps = np.real(s_fft * np.conj(s_fft)) / len(s_fft)

In [ ]:
s_ps.shape

In [ ]:
window_length = 20
rw = rolling_window(s_ps, window_length, window_length)

center = int(0.5 * window_size)
rolling_mean = []
rolling_median = []
rolling_mean_c = []
rolling_median_c = []

for i, window in enumerate(rw):
    rolling_mean.append(window.mean())
    rolling_median.append(np.median(window))
    rolling_mean_c.append(window.mean())
    rolling_median_c.append(np.median(window))


plt.figure()
fig, axs = plt.subplots(1, 1, figsize=(48,12))

# x_s_ps = np.linspace(0, 1, len())
plt.loglog(s_ps, label='Signal')
plt.loglog(rolling_mean, label='Mean Centered')
plt.plot(rolling_median, label='Median Centered')

plt.plot(rolling_mean_c, label='Mean Causal')
plt.plot(rolling_median_c, label='Median Causal')
plt.legend()
plt.show()

In [ ]:
s_ps.shape

In [ ]:
signal = np.array(x_g[0, :])

s_fft = np.fft.fft(signal)

s_ps = s_fft * np.conj(s_fft) # / len(s_fft)

threshold = 0.2 * s_ps.max()
psd_idxs = s_ps > threshold #array of 0 and 1
psd_clean = s_ps * psd_idxs #zero out all the unnecessary powers
fhat_clean = psd_idxs * s_fft #used to retrieve the signal
freq = (1 / (dt * len(s_fft))) * np.arange(len(s_fft)) #frequency array
idxs_half = np.arange(1, np.floor(len(s_fft)/2), dtype=np.int32) 

signal_filtered = np.fft.ifft(fhat_clean)


minsignal = -1
maxsignal = 1
fig, ax = plt.subplots(4,1)
ax[0].plot(tt, signal, color='b', lw=0.5, label='Noisy Signal')
# ax[0].plot(tt, signal_clean, color='r', lw=1, label='Clean Signal')
ax[0].set_ylim([minsignal, maxsignal])
ax[0].set_xlabel('t axis')
ax[0].set_ylabel('Vals')
ax[0].legend()

ax[1].plot(freq[idxs_half], np.abs(s_ps[idxs_half]), color='b', lw=0.5, label='PSD noisy')
ax[1].set_xlabel('Frequencies in Hz')
ax[1].set_ylabel('Amplitude')
ax[1].legend()

ax[2].plot(freq[idxs_half], np.abs(psd_clean[idxs_half]), color='r', lw=0.5, label='PSD clean')
ax[2].set_xlabel('Frequencies in Hz')
ax[2].set_ylabel('Amplitude')
ax[2].legend()

ax[3].plot(tt, signal_filtered, color='r', lw=0.5, label='Clean Signal Retrieved')
#ax[3].set_ylim([minsignal, maxsignal])
ax[3].set_xlabel('t axis')
ax[3].set_ylabel('Vals')
ax[3].legend()

plt.subplots_adjust(hspace=0.4)
plt.show()

In [ ]:
t_max = 50

n_ts = int(t_max / dt)
tt = dt * np.arange(n_ts)

window_length = 20
half_wl = int(window_length / 2)    
mean_locs = np.arange(half_wl, n_ts / 2, window_length)

for i in range(10):
    
    log_signal = np.log(np.abs(np.array(x_g[i, :])))
    signal = np.array(x_g[i, :])

    fig, ax = plt.subplots(3, 1, figsize=(15, 15))
    ax[0].plot(tt, signal, color='b', lw=0.5, label='Signal')
    # ax[0].plot(tt, signal_clean, color='r', lw=1, label='Clean Signal')
    # ax[0].set_ylim([-1, 1])
    ax[0].set_xlabel('Time [sec]')
    ax[0].set_ylabel('Normalized Amplitude')
    ax[0].legend()

    window_length = 20
    conv_data = np.convolve(log_signal, np.ones(window_length), 'valid') / window_length

    ax[1].plot(tt, log_signal, color='b', lw=0.5, label='Log Transformed Signal')
    ax[1].plot(tt[window_length-1:], conv_data, label='Rolling Mean')
    ax[1].set_xlabel('Time [sec]')
    ax[1].set_ylabel('Log Amplitude')
    ax[1].legend()

    s_fft = np.fft.rfft(signal)

    s_ps = s_fft * np.conj(s_fft)

    
    freq = np.fft.rfftfreq(signal.size, d=dt)

    rw = rolling_window(s_ps, window_length, window_length)

    center = int(0.5 * window_length)
    rolling_mean = [] # [None] * center

    for i, window in enumerate(rw):
        rolling_mean.append(window.mean())

    ax[2].loglog(freq, s_ps, color='r', lw=0.5, label='Fourier Transformed Signal')
    ax[2].plot(mean_locs, rolling_mean, label='Centered Rolling Mean')
    ax[2].set_xlabel('Frequenciy [Hz]')
    ax[2].set_ylabel('Fourier Amplitude')
    ax[2].set_xlim((0.1, 50))
    ax[2].legend()

    plt.subplots_adjust(hspace=0.4)
    plt.title('')
    plt.show()

    break

In [ ]:
dist = 60
lt = 1000

tt = dt * np.arange(lt)

for j in range(5):
    log_signal = np.log(np.abs(np.array(x_g[j, :])))
    signal = np.array(x_g[j, :])

    fig, ax = plt.subplots(4, 1, figsize=(15, 15))

    ax[0].plot(tt, signal, lw=0.5, label='Signal')
    # ax[0].plot(tt, signal_clean, color='r', lw=1, label='Clean Signal')
    ax[0].set_ylim([-1, 1])
    ax[0].set_xlabel('Time [s]')
    ax[0].set_ylabel('Normalized Amplitude')
    ax[0].legend()

    window_length = 40
    half_wl = int(window_length / 2)
    mean_data = np.convolve(log_signal, np.ones(window_length), 'valid') / window_length
    
    rw = rolling_window(log_signal, window_length, window_length)

    rolling_median = []
    for i, window in enumerate(rw):
        rolling_median.append(np.median(window))

    ax[1].plot(tt, log_signal, lw=0.5, label='Log Transformed Signal')
    ax[1].plot(tt[half_wl:-half_wl+1], mean_data, label='Centered Rolling Mean')
    # ax[1].plot(, rolling_median, label='Centered Rolling Median')
    ax[1].set_xlabel('Time [s]')
    ax[1].set_ylabel('Log Amplitude')
    ax[1].legend()

    rolling_max = [None] * half_wl
    rw = rolling_window(log_signal, window_length, window_length)

    for i, window in enumerate(rw):
        rolling_max.append(window.max())

    ax[2].plot(tt, log_signal, lw=0.5, label='Log Transformed Signal')
    ax[2].plot(tt[half_wl:], rolling_max, label='Rolling Max')
    ax[2].set_xlabel('Time [s]')
    ax[2].set_ylabel('Log Amplitude')
    ax[2].legend()

    s_fft = np.fft.rfft(signal)
    s_ps = np.real(s_fft * np.conj(s_fft))
    freq = np.fft.rfftfreq(signal.size, d=dt)

    rw = rolling_window(s_ps, window_length, window_length)
    rolling_mean = [None] * half_wl

    for i, window in enumerate(rw):
        rolling_mean.append(window.mean())

    ax[3].loglog(freq, s_ps, lw=0.5, label='Fourier Transformed Signal')
    mean_locs = np.arange(half_wl, lt, window_length)
    ax[3].plot(mean_locs, rolling_mean, label='Centered Rolling Mean')
    ax[3].set_xlabel('Frequency [Hz]')
    ax[3].set_ylabel('Fourier Amplitude')
    ax[3].set_xlim((0.1, 50))
    ax[3].legend()

    fig.suptitle(f'Random Synthetic Waveform. Dist: {dist}')
    plt.subplots_adjust(hspace=0.4)
    plt.tight_layout()

    fig_file = os.path.join(fig_dir, f'syntetic_data_plot_{j}.png')
    plt.savefig(fig_file, format='png')

In [ ]:
n_ts = int(t_max / dt)
tt = dt * np.arange(n_ts)
    
for i in range(10):
    
    log_signal = np.log(np.abs(np.array(x_g[i, :])))
    signal = np.array(x_g[i, :])

    plt.plot(tt, log_signal, label='Log-transformed Signal')
    plt.ylabel('Log-amplitude')
    plt.xlabel('Time [sec]')
    plt.legend()
    plt.show()

    plt.plot(tt, signal, label='Signal')
    plt.ylim((-1, 1))
    plt.ylabel('Normalized Amplitude')
    plt.xlabel('Time [sec]')
    plt.legend()
    plt.show()
    
    s_fft = np.fft.rfft(signal)
    s_ps = s_fft * np.conj(s_fft)
    
    freq = np.fft.rfftfreq(signal.size, d=dt)

    window_length = 20
    rw = rolling_window(s_ps, window_length, window_length)

    center = int(0.5 * window_length)
    rolling_mean = [None] * center

    for i, window in enumerate(rw):
        rolling_mean.append(window.mean())
    
    plt.loglog(freq, s_ps, label='Fourier Transform')
    plt.plot(freq[center:-center], rolling_mean[:-center], label='Centered Rolling Mean')
    plt.title('Fourier Transform of Signal')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Fourier Amplitude')
    plt.xlim((0.1, 50))
    plt.legend()
    plt.show()